## Imports 

In [1]:
import tensorflow as tf
import numpy as np

## Custom Layer with Weights

In [12]:
from tensorflow.keras.layers import Layer

class SimpleDense(Layer):
    
    def __init__(self, units=32):
        '''initializes the instance attributes'''
        super().__init__()
        self.units = units
        
    def build(self, input_shape):
        '''Create the states of the layer'''
        # initialize the weights
        init_w = tf.random_normal_initializer()
        self.w = tf.Variable(name="weights",
                            initial_value=init_w(shape=(input_shape[-1], self.units),
                                           dtype='float32'),
                            trainable=True)
                             
        # initialize the bias
        init_b = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
                            initial_value=init_b(shape=(self.units), dtype='float32'),
                            trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [13]:
my_dense = SimpleDense(units=1)

x = tf.ones((1, 1))
y = my_dense(x)

print(my_dense.variables)

[<tf.Variable 'simple_dense_6/weights:0' shape=(1, 1) dtype=float32, numpy=array([[0.01401721]], dtype=float32)>, <tf.Variable 'simple_dense_6/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


Testing the layer in a simple network

In [14]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model = tf.keras.Sequential([SimpleDense(units=1)])

model.compile(optimizer="SGD", loss="mean_squared_error")
model.fit(xs, ys, epochs=500, verbose=0)

# perform inference
print(model.predict([10.0]))

# check the updated state of the variables
print(model.variables)

[[18.981256]]
[<tf.Variable 'sequential_2/simple_dense_7/weights:0' shape=(1, 1) dtype=float32, numpy=array([[1.9972833]], dtype=float32)>, <tf.Variable 'sequential_2/simple_dense_7/bias:0' shape=(1,) dtype=float32, numpy=array([-0.99157715], dtype=float32)>]


## Adding Activation to the Custom Layer

In [18]:
class SimpleDense(Layer):
    def __init__(self, units=32, activation=None):
        super(SimpleDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel", 
                             initial_value=w_init(shape=(input_shape[-1], self.units),
                                            dtype="float32"),
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
                             initial_value=b_init(shape=(self.units),
                                                 dtype="float32"),
                             trainable=True)
        super().build(input_shape)
        
        
    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [20]:
mnist = tf.keras.datasets.mnist
(train_x, train_y), (test_x, test_y) = mnist.load_data()
train_x, test_x = train_x / 255.0, test_x / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    SimpleDense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_x, train_y, epochs=5)
model.evaluate(test_x, test_y)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.2996 - accuracy: 0.9135
Epoch 2/5
60000/60000 [==============================] - 4s 74us/sample - loss: 0.1433 - accuracy: 0.9572
Epoch 3/5
60000/60000 [==============================] - 4s 75us/sample - loss: 0.1081 - accuracy: 0.9683
Epoch 4/5
60000/60000 [==============================] - 4s 75us/sample - loss: 0.0880 - accuracy: 0.9726
Epoch 5/5
10000/10000 [==============================] - 0s 26us/sample - loss: 0.0744 - accuracy: 0.9759


[0.07438514485969208, 0.9759]